In [1]:
import os
import geopandas as gpd 
import pandas as pd

In [2]:
data_dir = "../../data/"

In [3]:
# Load the DE Africa Waterbodies Historical Extent Product.
deafrica_waterbodies_fp =  os.path.join(data_dir, "waterbodies.parquet")
deafrica_waterbodies = gpd.read_parquet(deafrica_waterbodies_fp)
print(f"Loaded {len(deafrica_waterbodies)} DE Africa waterbodies")

Loaded 700712 DE Africa waterbodies


In [4]:
cglrsed_fp = os.path.join(data_dir, "GLRSED_shp_V1.2_by continent/AF.shp")
cglrsed  = gpd.read_file(cglrsed_fp)
print(f"Loaded {len(cglrsed)} waterbodies")

Loaded 84059 waterbodies


In [5]:
assert deafrica_waterbodies.crs.equals(cglrsed.crs)

In [6]:
# Identify the resevoirs in the dataset
reservoirs = cglrsed[cglrsed["Reservoir"]==1]
print(f"Found {len(reservoirs)} artificial waterbodies")

Found 71321 artificial waterbodies


In [7]:
# How many waterbodies in the dataset have names?
filtered_by_name = cglrsed[cglrsed["Name"].notna()]
print(f"Found {len(filtered_by_name)} named waterbodies")

named_reservoirs = filtered_by_name[filtered_by_name["Reservoir"]==1]
print(f"Found {len(named_reservoirs)} named artificial waterbodies")

named_natural_waterbodies = filtered_by_name[filtered_by_name["Reservoir"]!=1]
print(f"Found {len(named_natural_waterbodies)} named natural waterbodies")

Found 112 named waterbodies
Found 45 named artificial waterbodies
Found 67 named natural waterbodies


In [8]:
# Identify the DE Africa waterbodies that are reservoirs based on intersection with the CGLRSED dataset
reservoir_uids = deafrica_waterbodies.sjoin(reservoirs, how="inner", predicate="intersects")["uid"].unique()
# Set the default to 0 which means is not a reservoir
deafrica_waterbodies["GLRSED_Reservoir"] = 0 
deafrica_waterbodies.loc[deafrica_waterbodies["uid"].isin(reservoir_uids), "GLRSED_Reservoir"] = 1

print(f"{len(deafrica_waterbodies[deafrica_waterbodies['GLRSED_Reservoir'] == 1])} DE Africa waterbodies identified as reservoirs")

24179 DE Africa waterbodies identified as reservoirs


In [9]:
# Assign names to the DE Africa waterbodies based on intersection with the CGLRSED dataset
joined = deafrica_waterbodies.sjoin(filtered_by_name[['Name',"geometry"]], how="left", predicate="intersects")
keep_columns = list(deafrica_waterbodies.columns) + ["Name"]
joined = joined[keep_columns]

In [10]:
# Get the waterbodies that got assigned more than one name
duplicates = joined[joined.duplicated("uid", keep=False)]

# Combine the Name for waterbodies with more than one Name.
name_combined = (
    duplicates
    .groupby('uid')['Name']
    .agg(lambda x: ', '.join(sorted(set(x))))
    .reset_index()
)
duplicates = duplicates.drop_duplicates(subset=['uid'], keep="first").drop(columns=["Name"]).merge(name_combined, on="uid")
duplicates

,uid,wb_id,area_m2,length_m,perim_m,geometry,GLRSED_Reservoir,Name
0,krvt190zcz,253769,1.525888e+10,1.386130e+06,5.533158e+07,"POLYGON ((14.92285 -4.66587, 14.92254 -4.66587...",1,"Mai-Ndombe, Tumba"
1,kvbe9t2dk9,341715,2.987207e+10,7.019634e+05,2.964780e+06,"POLYGON ((34.82889 -13.36661, 34.8292 -13.3666...",1,"Malawi, Malombe"
2,sewvr0derz,668141,9.156798e+09,1.680781e+06,2.620266e+07,"POLYGON ((30.33167 20.75167, 30.33198 20.75167...",1,"Jebel Aulia Reservoir, Nasser"


In [12]:
# Remove the waterbodies with more than one name from the larger dataframe. 
joined = joined.drop_duplicates(subset=['uid'], keep=False)
# Add the fixed waterbodies back. 
named_deafrica_waterbodies = pd.concat([joined, duplicates], ignore_index=True, axis=0).rename(columns={'Name': 'GLRSED_Name'})
named_deafrica_waterbodies

,uid,wb_id,area_m2,length_m,perim_m,geometry,GLRSED_Reservoir,GLRSED_Name
0,keyujdhemd,169502,7.812900e+06,2.124612e+04,8.256000e+04,"POLYGON ((32.2308 -23.10242, 32.23204 -23.1024...",0,NaN
1,ebgw0xw9gs,15400,1.710000e+04,4.500000e+02,1.020000e+03,"POLYGON ((-6.29716 5.31661, -6.29685 5.31661, ...",0,NaN
2,ebuyyf8nd3,15522,1.170000e+04,1.800000e+02,5.400000e+02,"POLYGON ((-4.27398 5.42099, -4.27398 5.41958, ...",0,NaN
3,ebytd6mkpw,16455,6.300000e+03,1.423025e+02,4.200000e+02,"POLYGON ((-2.0036 5.19902, -2.0036 5.19854, -2...",0,NaN
4,ebgrm39xyg,15303,6.551100e+06,1.647146e+04,8.562000e+04,"POLYGON ((-6.50269 5.55493, -6.50238 5.55493, ...",0,NaN
...,...,...,...,...,...,...,...,...
700707,t4jtnvcjmy,700711,9.900000e+03,1.800000e+02,5.400000e+02,"POLYGON ((53.03262 12.16179, 53.03355 12.16179...",0,NaN
700708,t4jtpnq7kg,700712,1.260000e+04,2.100000e+02,6.000000e+02,"POLYGON ((53.04972 12.16419, 53.04972 12.16395...",0,NaN
700709,krvt190zcz,253769,1.525888e+10,1.386130e+06,5.533158e+07,"POLYGON ((14.92285 -4.66587, 14.92254 -4.66587...",1,"Mai-Ndombe, Tumba"
700710,kvbe9t2dk9,341715,2.987207e+10,7.019634e+05,2.964780e+06,"POLYGON ((34.82889 -13.36661, 34.8292 -13.3666...",1,"Malawi, Malombe"


In [13]:
print(f"{len(named_deafrica_waterbodies[named_deafrica_waterbodies["GLRSED_Name"].notna()])} DE Africa waterbodies assigned name")

6850 DE Africa waterbodies assigned name


In [14]:
assert len(deafrica_waterbodies) == len(named_deafrica_waterbodies)

In [15]:
# Export the updated waterbodies
named_deafrica_waterbodies.to_parquet(os.path.join(data_dir, "deafrica_waterbodies_cglrsed_update.parquet"))